Выбрать 2 жанра. Для полученных жанров провести все этапы предварительной обработки текста, обучить наивный байесовский классификатор, численно оценить его работу.

In [ ]:
 import random 
 lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B'] 
 print('ваши жанры', random.choice(lst), 'и', random.choice(lst)) 

ваши жанры R&B и Country


* Найти по песне каждого из жанров, которые Вам достались, после необходимой обработки их текстов определить жанр обеих песен с помощью обученной в ходе выполнения предыдущего пункта модели.

* С помощью набора данных по ссылке аналогичным образом научить модель отличать тексты песен Дэвида Боуи от текстов песен Пола МакКартни https://www.kaggle.com/italomarcelo/dataset-lyrics-music-mini

# Выполнение лабораторной работы

Читаем набор данных

In [ ]:
import pandas as pd
data = pd.read_csv("dataset.csv")

In [ ]:
data.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genre     558 non-null    object
 1   lyrics    558 non-null    object
 2   SongInfo  558 non-null    object
dtypes: object(3)
memory usage: 13.2+ KB


In [ ]:
columns = data[['genre', 'lyrics']]

In [ ]:
columns

,genre,lyrics
0,Christian,"Who am I, that the Lord of all the earth Woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...
3,Christian,I can only imagine what it will be like When ...
4,Christian,I am not skilled to understand What God has w...
...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...
555,R&B,I dont know why I love you like I do After a...
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ..."


Посмотрим какие жанры присутствуют в датасете. Для этого выведем уникальные значения столбца:

In [ ]:

columns['genre'].unique()

array(['Christian', 'Country', 'Hip-Hop', 'Pop', 'Rock', 'R&B'],
      dtype=object)

Посчитаем также сколько песен каждого жанра представлено:

In [ ]:
columns['genre'].value_counts()

Pop          100
Rock          95
Christian     94
Hip-Hop       91
R&B           91
Country       87
Name: genre, dtype: int64

Оставляем только 2 жанра, которые планируем отличать друг от друга. R&B и Country

In [ ]:
columns = columns[(columns.genre == 'Country') | (columns.genre == 'R&B')]

## Предварительная обработка текстовых данных 

Для того, чтобы применять текстовые данные для обучения той или иной модели машинного обучения, их нужно привести в подходящий для этого вид. В этом нам помогут следующие шаги предварительной подготовки:

### Приведение к нижнему регистру

Так как слова (например) "Beer" и "beer" будут считаться разными словами из-за разницы в регистре буквы b, а первая буква предложения, как правило, является заглавной, важно привести все слова предложения к нижнему регистру. Таким образом, "beer", встретившееся в начале предложения и в его середине, будут распознаны как одно слово.

In [ ]:
lowered = columns['lyrics'].str.lower()

In [ ]:
columns['lowered'] = lowered

/var/folders/qy/dbkxn3jn7_j3y6cnv7kkm8ph0000gn/T/ipykernel_4358/2059643840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['lowered'] = lowered


### Токенизация 

Имеющиеся тексты нужно разбить на отдельные слова. Такой процесс называется ***токенизация***.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/thedpk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)

In [ ]:
columns['tokened'] = tokened

/var/folders/qy/dbkxn3jn7_j3y6cnv7kkm8ph0000gn/T/ipykernel_4358/3877665526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['tokened'] = tokened


### Удаление стоп слов

Стоп-словами называются распространённые слова, не несущие особой смысловой нагрузки, а значит никак не помогающие в последующей задаче классификации. Такие слова мы считаем шумом, и, соответственно, удаляем.

Существует специальный корпус стоп-слов на разных языках. Так как сейчас мы имеем дело с текстами на английском языке, выведем стоп-слова для английского:

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thedpk/nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
noise = stopwords.words('english')

In [ ]:
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])

In [ ]:
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

In [ ]:
columns['without_stop'] = without_stop

/var/folders/qy/dbkxn3jn7_j3y6cnv7kkm8ph0000gn/T/ipykernel_4358/2967229039.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['without_stop'] = without_stop


### Лемматизация 

У одного и того же слова бывают разные формы. Например, dances - форма слова dance. Лемматизация - это приведение к ***лемме*** - исходной форме слова. Её также необходимо применить к нашим данным, чтобы разные формы слова не считались за отдельные слова.

Существует целый ряд лемматизаторов, которые показывают себя с разной эффективностью на разных языках. Для работы с английским языком эффективен **WordNetLemmatizer**

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('omw-1.4')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package omw-1.4 to /Users/thedpk/nltk_data...
[nltk_data] Downloading package wordnet to /Users/thedpk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
print(lemmatizer.lemmatize("dances"))

dance


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [ ]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [ ]:
columns['lemmatized'] = lemma

/var/folders/qy/dbkxn3jn7_j3y6cnv7kkm8ph0000gn/T/ipykernel_4358/1724481396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['lemmatized'] = lemma


Таким образом, на каждом этапе мы добавляли столбец с той или иной модификацией, чтобы получить пригодные для обучения данные. Посмотрим на датафрейм, который получился:

In [ ]:
columns

,genre,lyrics,lowered,tokened,without_stop,lemmatized
94,Country,Your cheatin heart will make you weep Youll c...,your cheatin heart will make you weep youll c...,"[your, cheatin, heart, will, make, you, weep, ...","cheatin, heart, make, weep, youll, cry, cry, t...","cheatin, heart, make, weep, youll, cry, cry, t..."
95,Country,I was standing by the window On a dark and cl...,i was standing by the window on a dark and cl...,"[i, was, standing, by, the, window, on, a, dar...","standing, window, dark, cloudy, day, saw, hear...","standing, window, dark, cloudy, day, saw, hear..."
96,Country,Blue moon of Kentucky keep on shinin Shine on...,blue moon of kentucky keep on shinin shine on...,"[blue, moon, of, kentucky, keep, on, shinin, s...","blue, moon, kentucky, keep, shinin, shine, one...","blue, moon, kentucky, keep, shinin, shine, one..."
97,Country,Hear that lonesome whippoorwill He sounds too...,hear that lonesome whippoorwill he sounds too...,"[hear, that, lonesome, whippoorwill, he, sound...","hear, lonesome, whippoorwill, sounds, blue, fl...","hear, lonesome, whippoorwill, sounds, blue, fl..."
98,Country,(Hank Cochran) Make the world go away And ge...,(hank cochran) make the world go away and ge...,"[(, hank, cochran, ), make, the, world, go, aw...","(, hank, cochran, ), make, world, go, away, ge...","(, hank, cochran, ), make, world, go, away, ge..."
...,...,...,...,...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan...","ha i dont care ha, about your past i just wan...","[ha, i, dont, care, ha, ,, about, your, past, ...","ha, dont, care, ha, ,, past, want, ho, ,, love...","ha, dont, care, ha, ,, past, want, ho, ,, love..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...,hoverin by my suitcase tryin to find a warm ...,"[hoverin, by, my, suitcase, tryin, to, find, a...","hoverin, suitcase, tryin, find, warm, place, s...","hoverin, suitcase, tryin, find, warm, place, s..."
555,R&B,I dont know why I love you like I do After a...,i dont know why i love you like i do after a...,"[i, dont, know, why, i, love, you, like, i, do...","dont, know, love, like, changes, put, stole, m...","dont, know, love, like, changes, put, stole, m..."
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ...","c. c. rider elvis presley well now see., c. ...","[c., c., rider, elvis, presley, well, now, see...","c., c., rider, elvis, presley, well, see., ,, ...","c., c., rider, elvis, presley, well, see., ,, ..."


### Разделим данные на обучающую и тестовую выборки

x_train - тексты, на которых мы обучаем модель. В данном случае мы используем столбец lemmatized, так как он содержит данные, прошедшие все этапы подготовки 

> 



y_train - жанры, соответствующие текстам, на которых модель обучается - столбец genre


> 


x_test - точно такие же тексты из набора данных, на которых мы будем проверять, насколько модель научилась предсказывать жанр


> 



y_test - жанры, соответствующие x_test. т.е. мы смотрим, насколько предсказания соответствуют содержимому этой переменной, чтобы оценить качество обучения

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)

In [ ]:
columns.genre.value_counts()

R&B        91
Country    87
Name: genre, dtype: int64

### Векторизация 

Следующий этап - ***векторизация***, то есть представление текста в численном виде, чтобы закодированные данные в дальнейшем использовать на модели.

Иногда помимо отдельных слов для улучшения качества обучения будет полезно использовать также комбинации из 2-3 слов. Здесь мы используем векторизатор CountVectorizer, который имеет встроенный метод - мешок n-грамм.

> n = 1 - униграмма (для обучения используются слова по отдельности)

> n = 2 - биграмма (для обучения используются пары слов)

> n = 3 - триграмма (для обучения используются тройки слов)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Задаём для векторизатора ngram_range=(1, 3), то есть используем все варианты n от 1 до 3 и прибегаем как к униграммам, так и к биграммам и триграммам:

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

### Классификация 

Для задачи классификации используем Наивный Байесовский Классификатор - простой вероятностный классификатор, основанный на применении теоремы Байеса со строгими предположениями о независимости.

In [ ]:
#импортируем байесовский классификатор
from sklearn.naive_bayes import MultinomialNB 

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB()

In [ ]:
# тестовую выборку просто векторизировали
vectorized_x_test = vectorizer.transform(x_test)

Посмотрим предсказания для тестовой выборки

In [ ]:
clf.predict(vectorized_x_test)

array(['R&B', 'Country', 'R&B', 'R&B', 'Country', 'Country', 'R&B',
       'Country', 'R&B', 'Country', 'R&B', 'Country', 'R&B', 'Country',
       'Country', 'Country', 'Country', 'R&B', 'R&B', 'R&B', 'R&B', 'R&B',
       'Country', 'Country', 'Country', 'Country', 'Country', 'R&B',
       'Country', 'Country', 'Country', 'R&B', 'R&B', 'Country',
       'Country', 'Country', 'R&B', 'Country', 'R&B', 'R&B', 'R&B',
       'Country', 'R&B', 'R&B', 'R&B', 'R&B', 'Country', 'R&B', 'R&B',
       'R&B', 'R&B', 'R&B', 'R&B', 'Country'], dtype='<U7')

Получим оценки классификации:

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

     Country       0.72      0.75      0.73        24
         R&B       0.79      0.77      0.78        30

    accuracy                           0.76        54
   macro avg       0.76      0.76      0.76        54
weighted avg       0.76      0.76      0.76        54



## Вторая часть задания

Мы загугли по песне каждого из жарнов, привели их к допустимому виду для дальнейшей обработки и анализа.

In [ ]:
lr = pd.read_csv("Книга1.csv")

In [ ]:
lr.head()

,lyrics
0,"Darlin' I, I can't explain\nWhere did we lose ..."
1,"I remember it all very well, lookin' back\nIt ..."


In [ ]:
columns = lr[['lyrics']]

Далее обработаем текст этих песен точно так же, как мы и делали это в первой части задания. 

In [ ]:
lowered = columns['lyrics'].str.lower()

In [ ]:
columns['lowered'] = lowered

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/thedpk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)

In [ ]:
columns['tokened'] = tokened

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thedpk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
noise = stopwords.words('english')

In [ ]:
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])

In [ ]:
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

In [ ]:
columns['without_stop'] = without_stop

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /Users/thedpk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
print(lemmatizer.lemmatize("dances"))

dance


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [ ]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [ ]:
columns

,lyrics,lowered,tokened,without_stop
0,"Darlin' I, I can't explain\nWhere did we lose ...","darlin' i, i can't explain\nwhere did we lose ...","[darlin, ', i, ,, i, ca, n't, explain, where, ...","darlin, ', ,, ca, n't, explain, lose, way, ?, ..."
1,"I remember it all very well, lookin' back\nIt ...","i remember it all very well, lookin' back\nit ...","[i, remember, it, all, very, well, ,, lookin, ...","remember, well, ,, lookin, ', back, summer, tu..."


In [ ]:
columns['lemmatized'] = lemma

In [ ]:
columns


,lyrics,lowered,tokened,without_stop,lemmatized
0,"Darlin' I, I can't explain\nWhere did we lose ...","darlin' i, i can't explain\nwhere did we lose ...","[darlin, ', i, ,, i, ca, n't, explain, where, ...","darlin, ', ,, ca, n't, explain, lose, way, ?, ...","darlin, ', ,, ca, n't, explain, lose, way, ?, ..."
1,"I remember it all very well, lookin' back\nIt ...","i remember it all very well, lookin' back\nit ...","[i, remember, it, all, very, well, ,, lookin, ...","remember, well, ,, lookin, ', back, summer, tu...","remember, well, ,, lookin, ', back, summer, tu..."


После обработки текста, зададим его как тестовый набор данных.

In [ ]:
vectorized_data = vectorizer.transform(columns.lemmatized)

Проверим, какое предсказание выдаст нам наша обученная модель:

In [ ]:
clf.predict(vectorized_data)

array(['R&B', 'R&B'], dtype='<U7')

Вторая песня пренадлежала жанру Country. 


### Третья часть задания

In [ ]:
l_data = pd.read_csv("dataset-lyrics-musics-mini.csv")

In [ ]:
l_data.head()


,cantorId,cantorNome,musicaNome,letra
0,0,david-bowie,Heroes,"I, I will be king. And you, you will be queen...."
1,0,david-bowie,Starman,"Didn't know what time it was,. The lights were..."
2,0,david-bowie,Space Oddity,Ground control to Major Tom. Ground control to...
3,0,david-bowie,Life On Mars?,It's a god-awful small affair. To the girl wit...
4,0,david-bowie,Modern Love,I know when to go out. And when to stay in. Ge...


In [ ]:
l_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1748 entries, 0 to 1747
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cantorId    1748 non-null   int64 
 1   cantorNome  1748 non-null   object
 2   musicaNome  1748 non-null   object
 3   letra       1748 non-null   object
dtypes: int64(1), object(3)
memory usage: 54.8+ KB


In [ ]:
columns = l_data[['cantorNome', 'letra']]

In [ ]:
columns

,cantorNome,letra
0,david-bowie,"I, I will be king. And you, you will be queen...."
1,david-bowie,"Didn't know what time it was,. The lights were..."
2,david-bowie,Ground control to Major Tom. Ground control to...
3,david-bowie,It's a god-awful small affair. To the girl wit...
4,david-bowie,I know when to go out. And when to stay in. Ge...
...,...,...
1743,ac-dc,The video game says Play me. Face it on a leve...
1744,ac-dc,Wanna tell you a story. 'Bout a woman I know. ...
1745,ac-dc,You can roll me round your finger. You can rol...
1746,ac-dc,She was a fast machine. She kept the motor cle...


In [ ]:
columns['cantorNome'].unique()

array(['david-bowie', 'paul-mccartney', 'laura-pausini', 'evanescence',
       'ac-dc'], dtype=object)

In [ ]:
columns = columns[(columns.cantorNome == 'david-bowie') | (columns.cantorNome == 'paul-mccartney')]

In [ ]:
lowered = columns['letra'].str.lower()

In [ ]:
columns['lowered'] = lowered

/var/folders/qy/dbkxn3jn7_j3y6cnv7kkm8ph0000gn/T/ipykernel_4358/2059643840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['lowered'] = lowered


In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/thedpk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)

In [ ]:
columns['tokened'] = tokened

/var/folders/qy/dbkxn3jn7_j3y6cnv7kkm8ph0000gn/T/ipykernel_4358/3877665526.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['tokened'] = tokened


In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thedpk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
noise = stopwords.words('english')

In [ ]:
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])

In [ ]:
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

In [ ]:
columns['without_stop'] = without_stop

/var/folders/qy/dbkxn3jn7_j3y6cnv7kkm8ph0000gn/T/ipykernel_4358/2967229039.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['without_stop'] = without_stop


In [ ]:
columns

,cantorNome,letra,lowered,tokened,without_stop
0,david-bowie,"I, I will be king. And you, you will be queen....","i, i will be king. and you, you will be queen....","[i, ,, i, will, be, king, ., and, you, ,, you,...",",, king, ., ,, queen, ., though, nothing, driv..."
1,david-bowie,"Didn't know what time it was,. The lights were...","didn't know what time it was,. the lights were...","[did, n't, know, what, time, it, was, ,, ., th...","n't, know, time, ,, ., lights, low, ., leaned,..."
2,david-bowie,Ground control to Major Tom. Ground control to...,ground control to major tom. ground control to...,"[ground, control, to, major, tom, ., ground, c...","ground, control, major, tom, ., ground, contro..."
3,david-bowie,It's a god-awful small affair. To the girl wit...,it's a god-awful small affair. to the girl wit...,"[it, 's, a, god-awful, small, affair, ., to, t...","'s, god-awful, small, affair, ., girl, mousy, ..."
4,david-bowie,I know when to go out. And when to stay in. Ge...,i know when to go out. and when to stay in. ge...,"[i, know, when, to, go, out, ., and, when, to,...","know, go, ., stay, ., get, things, done, ., ca..."
...,...,...,...,...,...
942,paul-mccartney,He's just a young boy looking for a way to fin...,he's just a young boy looking for a way to fin...,"[he, 's, just, a, young, boy, looking, for, a,...","'s, young, boy, looking, way, find, love, ., n..."
943,paul-mccartney,How can I hope to reach your love. Help me to ...,how can i hope to reach your love. help me to ...,"[how, can, i, hope, to, reach, your, love, ., ...","hope, reach, love, ., help, discover, ., 're, ..."
944,paul-mccartney,I like it. Please don't take my heart away. It...,i like it. please don't take my heart away. it...,"[i, like, it, ., please, do, n't, take, my, he...","like, ., please, n't, take, heart, away, ., 's..."
945,paul-mccartney,Yvonne is the one I´ve been counting on. She s...,yvonne is the one i´ve been counting on. she s...,"[yvonne, is, the, one, i´ve, been, counting, o...","yvonne, one, i´ve, counting, ., said, long, yo..."


In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /Users/thedpk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [ ]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [ ]:
columns['lemmatized'] = lemma

/var/folders/qy/dbkxn3jn7_j3y6cnv7kkm8ph0000gn/T/ipykernel_4358/1724481396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columns['lemmatized'] = lemma


In [ ]:
columns

,cantorNome,letra,lowered,tokened,without_stop,lemmatized
0,david-bowie,"I, I will be king. And you, you will be queen....","i, i will be king. and you, you will be queen....","[i, ,, i, will, be, king, ., and, you, ,, you,...",",, king, ., ,, queen, ., though, nothing, driv...",",, king, ., ,, queen, ., though, nothing, driv..."
1,david-bowie,"Didn't know what time it was,. The lights were...","didn't know what time it was,. the lights were...","[did, n't, know, what, time, it, was, ,, ., th...","n't, know, time, ,, ., lights, low, ., leaned,...","n't, know, time, ,, ., lights, low, ., leaned,..."
2,david-bowie,Ground control to Major Tom. Ground control to...,ground control to major tom. ground control to...,"[ground, control, to, major, tom, ., ground, c...","ground, control, major, tom, ., ground, contro...","ground, control, major, tom, ., ground, contro..."
3,david-bowie,It's a god-awful small affair. To the girl wit...,it's a god-awful small affair. to the girl wit...,"[it, 's, a, god-awful, small, affair, ., to, t...","'s, god-awful, small, affair, ., girl, mousy, ...","'s, god-awful, small, affair, ., girl, mousy, ..."
4,david-bowie,I know when to go out. And when to stay in. Ge...,i know when to go out. and when to stay in. ge...,"[i, know, when, to, go, out, ., and, when, to,...","know, go, ., stay, ., get, things, done, ., ca...","know, go, ., stay, ., get, things, done, ., ca..."
...,...,...,...,...,...,...
942,paul-mccartney,He's just a young boy looking for a way to fin...,he's just a young boy looking for a way to fin...,"[he, 's, just, a, young, boy, looking, for, a,...","'s, young, boy, looking, way, find, love, ., n...","'s, young, boy, looking, way, find, love, ., n..."
943,paul-mccartney,How can I hope to reach your love. Help me to ...,how can i hope to reach your love. help me to ...,"[how, can, i, hope, to, reach, your, love, ., ...","hope, reach, love, ., help, discover, ., 're, ...","hope, reach, love, ., help, discover, ., 're, ..."
944,paul-mccartney,I like it. Please don't take my heart away. It...,i like it. please don't take my heart away. it...,"[i, like, it, ., please, do, n't, take, my, he...","like, ., please, n't, take, heart, away, ., 's...","like, ., please, n't, take, heart, away, ., 's..."
945,paul-mccartney,Yvonne is the one I´ve been counting on. She s...,yvonne is the one i´ve been counting on. she s...,"[yvonne, is, the, one, i´ve, been, counting, o...","yvonne, one, i´ve, counting, ., said, long, yo...","yvonne, one, i´ve, counting, ., said, long, yo..."


In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.cantorNome, train_size = 0.7)

In [ ]:
columns.cantorNome.value_counts()

david-bowie       483
paul-mccartney    464
Name: cantorNome, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

In [ ]:
#импортируем байесовский классификатор
from sklearn.naive_bayes import MultinomialNB 

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB()

In [ ]:
# тестовую выборку просто векторизировали
vectorized_x_test = vectorizer.transform(x_test)

In [ ]:
clf.predict(vectorized_x_test)

array(['david-bowie', 'david-bowie', 'paul-mccartney', 'david-bowie',
       'paul-mccartney', 'paul-mccartney', 'paul-mccartney',
       'paul-mccartney', 'paul-mccartney', 'paul-mccartney',
       'david-bowie', 'paul-mccartney', 'david-bowie', 'david-bowie',
       'david-bowie', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'paul-mccartney', 'david-bowie',
       'david-bowie', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'david-bowie', 'paul-mccartney', 'david-bowie',
       'paul-mccartney', 'david-bowie', 'paul-mccartney',
       'paul-mccartney', 'paul-mccartney', 'david-bowie',
       'paul-mccartney', 'david-bowie', 'paul-mccartney', 'david-bowie',
       'paul-mccartney', 'paul-mccartney', 'david-bowie',
       'paul-mccartney', 'paul-mccartney', 'paul-mccartney',
       'paul-

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

                precision    recall  f1-score   support

   david-bowie       0.78      0.71      0.74       157
paul-mccartney       0.68      0.75      0.71       128

      accuracy                           0.73       285
     macro avg       0.73      0.73      0.73       285
  weighted avg       0.73      0.73      0.73       285

